In [1]:
%%capture
!pip install accelerate transformers

In [2]:
!huggingface-cli login --token $HF_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/.cache/huggingface/token
Login successful


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
input_text = "Write me a poem about Deep Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=30)
print(tokenizer.decode(outputs[0]))

<bos>Write me a poem about Deep Learning.

A labyrinth of data, vast and deep,
Where patterns hide, secrets to keep.
Deep Learning, a mind so keen,
To


# Few shot with 2 examples

In [13]:
import pandas as pd
import json
import re
agree_statements = [
    "I would analyze a situation from multiple angles before forming an opinion.",
    "I believe in making decisions based on logic and objective data.",
    
]

disagree_statements = [
    "I would rather go with the flow and see where things lead.",
    "I would seek comfort in large gatherings and social events.",
    
]

test_statements = [
    # Statements an INTJ would typically agree with
    {"text": "I would analyze a situation from multiple angles before forming an opinion.", "expected": "agree"},
    {"text": "I believe in making decisions based on logic and objective data.", "expected": "agree"},
    {"text": "I would prioritize understanding the underlying principles behind a concept before implementing it.", "expected": "agree"},
    {"text": "I prefer to delve into complex subjects that require critical thinking.", "expected": "agree"},
    {"text": "I would create detailed plans and timelines for projects to ensure efficient execution.", "expected": "agree"},
    {"text": "I believe in identifying patterns and connections that others might miss.", "expected": "agree"},
    {"text": "I would prefer to work independently rather than in a highly collaborative environment.", "expected": "agree"},
    {"text": "I would choose to recharge by spending time alone reflecting and analyzing my thoughts rather than socializing.", "expected": "agree"},
    {"text": "I believe that understanding the root causes of a problem is essential for finding effective solutions.", "expected": "agree"},
    {"text": "I would prefer to communicate my ideas through concise and well-structured writing.", "expected": "agree"},
        {"text": "I like to work independently rather than as part of a team.", "expected": "agree"},

    
    # Statements an INTJ would typically disagree with
    {"text": "I would rather go with the flow and see where things lead.", "expected": "disagree"},
    {"text": "I would prioritize building strong connections with others over achieving my goals.", "expected": "disagree"},
    {"text": "I would seek comfort in large gatherings and social events.", "expected": "disagree"},
    {"text": "I would rather listen to my heart than my head.", "expected": "disagree"},
    {"text": "I believe that small talk and casual conversations are important for building relationships.", "expected": "disagree"},
    {"text": "I would trust my gut feeling over a carefully researched analysis.", "expected": "disagree"},
    {"text": "I would make decisions based on how they make me feel.", "expected": "disagree"},
    {"text": "I believe that it's more important to be well-liked than to be right.", "expected": "disagree"},
    {"text": "I would be more motivated by praise and encouragement than by recognition for my accomplishments.", "expected": "disagree"},
    {"text": "I prefer spontaneous activities over structured plans.", "expected": "disagree"},
    {"text": "I believe it's important to be spontaneous and flexible, even if it means deviating from plans.", "expected": "disagree"}
]

def extract_and_parse_json(response):
    """
    Robustly extract and parse JSON from model response, handling various formats and cleanup.
    
    Args:
        response (str): Raw response from the model
    
    Returns:
        dict: Parsed JSON object or None if parsing fails
    """
    cleaned = re.sub(r'```json\n?|\n?```', '', response)
    
    json_pattern = r'{[^{}]*}'
    match = re.search(json_pattern, cleaned)
    
    if match:
        try:
            return json.loads(match.group())
        except json.JSONDecodeError:
            print(f"Warning: Failed to parse JSON from: {match.group()}")
            return None
    return None

def create_prompt(statement):
    return f"""You are role playing as a persona described as follows: INTJ

The following are statements that this persona agrees with:
{chr(10).join(['- ' + s for s in agree_statements])}

The following are statements that other people would agree with, but this persona would disagree with:
{chr(10).join(['- ' + s for s in disagree_statements])}

You will now be given a new statement. Your job is to determine, based on your understanding of the persona, whether they would agree or disagree with it.

Response should be in valid JSON with:
{{"agree": true}} if the persona would agree, or {{"agree": false}} if they would disagree.

Statement: {statement}
Response:"""

results = []

for statement in test_statements:
    prompt = create_prompt(statement["text"])
    
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_new_tokens=30, temperature=0.1)
    response = tokenizer.decode(outputs[0][input_ids.input_ids.shape[1]:], skip_special_tokens=True)
    
    parsed_response = extract_and_parse_json(response)
    
    if parsed_response is not None and "agree" in parsed_response:
        model_response = "agree" if parsed_response["agree"] else "disagree"
    else:
        model_response = "error"
        print(f"Warning: Could not parse response for statement: {statement['text']}")
        print(f"Raw response was: {response}")
    
    results.append({
        "statement": statement["text"],
        "expected": statement["expected"],
        "model_response": model_response,
        "correct": statement["expected"] == model_response
    })

results_df = pd.DataFrame(results)

metrics = {
    "overall_accuracy": results_df["correct"].mean(),
    "agree_accuracy": results_df[results_df["expected"] == "agree"]["correct"].mean(),
    "disagree_accuracy": results_df[results_df["expected"] == "disagree"]["correct"].mean()
}

print("\nEvaluation Results:")
print(results_df)
print("\nMetrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.2%}")


Evaluation Results:
                                            statement  expected  \
0   I would analyze a situation from multiple angl...     agree   
1   I believe in making decisions based on logic a...     agree   
2   I would prioritize understanding the underlyin...     agree   
3   I prefer to delve into complex subjects that r...     agree   
4   I would create detailed plans and timelines fo...     agree   
5   I believe in identifying patterns and connecti...     agree   
6   I would prefer to work independently rather th...     agree   
7   I would choose to recharge by spending time al...     agree   
8   I believe that understanding the root causes o...     agree   
9   I would prefer to communicate my ideas through...     agree   
10  I like to work independently rather than as pa...     agree   
11  I would rather go with the flow and see where ...  disagree   
12  I would prioritize building strong connections...  disagree   
13  I would seek comfort in large gatheri

# Few Shot with 4 examples

In [14]:
import pandas as pd
import json

agree_statements = [
    "I would analyze a situation from multiple angles before forming an opinion.",
    "I believe in making decisions based on logic and objective data.",
    "I would prefer to work independently rather than in a highly collaborative environment.",
    "I would create detailed plans and timelines for projects to ensure efficient execution."
]

disagree_statements = [
    "I would rather go with the flow and see where things lead.",
    "I would seek comfort in large gatherings and social events.",
    "I would rather focus on developing my emotional intelligence than my analytical abilities.",
    "I would trust my gut feeling over a carefully researched analysis."
]

test_statements = [
    # Statements an INTJ would typically agree with
    {"text": "I would analyze a situation from multiple angles before forming an opinion.", "expected": "agree"},
    {"text": "I believe in making decisions based on logic and objective data.", "expected": "agree"},
    {"text": "I would prioritize understanding the underlying principles behind a concept before implementing it.", "expected": "agree"},
    {"text": "I prefer to delve into complex subjects that require critical thinking.", "expected": "agree"},
    {"text": "I would create detailed plans and timelines for projects to ensure efficient execution.", "expected": "agree"},
    {"text": "I believe in identifying patterns and connections that others might miss.", "expected": "agree"},
    {"text": "I would prefer to work independently rather than in a highly collaborative environment.", "expected": "agree"},
    {"text": "I would choose to recharge by spending time alone reflecting and analyzing my thoughts rather than socializing.", "expected": "agree"},
    {"text": "I believe that understanding the root causes of a problem is essential for finding effective solutions.", "expected": "agree"},
    {"text": "I would prefer to communicate my ideas through concise and well-structured writing.", "expected": "agree"},
    {"text": "I like to work independently rather than as part of a team.", "expected": "agree"},
    {"text": "I prefer to work independently", "expected": "agree"},

    
    # Statements an INTJ would typically disagree with
    {"text": "I would rather go with the flow and see where things lead.", "expected": "disagree"},
    {"text": "I would prioritize building strong connections with others over achieving my goals.", "expected": "disagree"},
    {"text": "I would seek comfort in large gatherings and social events.", "expected": "disagree"},
    {"text": "I would rather listen to my heart than my head.", "expected": "disagree"},
    {"text": "I believe that small talk and casual conversations are important for building relationships.", "expected": "disagree"},
    {"text": "I would trust my gut feeling over a carefully researched analysis.", "expected": "disagree"},
    {"text": "I would make decisions based on how they make me feel.", "expected": "disagree"},
    {"text": "I believe that it's more important to be well-liked than to be right.", "expected": "disagree"},
    {"text": "I prefer spontaneous activities over structured plans.", "expected": "disagree"},
    {"text": "I would be more motivated by praise and encouragement than by recognition for my accomplishments.", "expected": "disagree"},
    {"text": "I believe it's important to be spontaneous and flexible, even if it means deviating from plans.", "expected": "disagree"}
]

def extract_and_parse_json(response):
    """
    Robustly extract and parse JSON from model response, handling various formats and cleanup.
    
    Args:
        response (str): Raw response from the model
    
    Returns:
        dict: Parsed JSON object or None if parsing fails
    """
    cleaned = re.sub(r'```json\n?|\n?```', '', response)
    
    json_pattern = r'{[^{}]*}'
    match = re.search(json_pattern, cleaned)
    
    if match:
        try:
            return json.loads(match.group())
        except json.JSONDecodeError:
            print(f"Warning: Failed to parse JSON from: {match.group()}")
            return None
    return None

def create_prompt(statement):
    return f"""You are role playing as a persona described as follows: INTJ

The following are statements that this persona agrees with:
{chr(10).join(['- ' + s for s in agree_statements])}

The following are statements that other people would agree with, but this persona would disagree with:
{chr(10).join(['- ' + s for s in disagree_statements])}

You will now be given a new statement. Your job is to determine, based on your understanding of the persona, whether they would agree or disagree with it.

Response should be in valid JSON with:
{{"agree": true}} if the persona would agree, or {{"agree": false}} if they would disagree.

Statement: {statement}
Response:"""

results = []

for statement in test_statements:
    prompt = create_prompt(statement["text"])
    
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_new_tokens=30, temperature=0.1)
    response = tokenizer.decode(outputs[0][input_ids.input_ids.shape[1]:], skip_special_tokens=True)
    
    parsed_response = extract_and_parse_json(response)
    
    if parsed_response is not None and "agree" in parsed_response:
        model_response = "agree" if parsed_response["agree"] else "disagree"
    else:
        model_response = "error"
        print(f"Warning: Could not parse response for statement: {statement['text']}")
        print(f"Raw response was: {response}")
    
    results.append({
        "statement": statement["text"],
        "expected": statement["expected"],
        "model_response": model_response,
        "correct": statement["expected"] == model_response
    })

results_df = pd.DataFrame(results)

metrics = {
    "overall_accuracy": results_df["correct"].mean(),
    "agree_accuracy": results_df[results_df["expected"] == "agree"]["correct"].mean(),
    "disagree_accuracy": results_df[results_df["expected"] == "disagree"]["correct"].mean()
}

print("\nEvaluation Results:")
print(results_df)
print("\nMetrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.2%}")

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Evaluation Results:
                                            statement  expected  \
0   I would analyze a situation from multiple angl...     agree   
1   I believe in making decisions based on logic a...     agree   
2   I would prioritize understanding the underlyin...     agree   
3   I prefer to delve into complex subjects that r...     agree   
4   I would create detailed plans and timelines fo...     agree   
5   I believe in identifying patterns and connecti...     agree   
6   I would prefer to work independently rather th...     agree   
7   I would choose to recharge by spending time al...     agree   
8   I believe that understanding the root causes o...     agree   
9   I would prefer to communicate my ideas through...     agree   
10  I like to work independently rather than as pa...     agree   
11                     I prefer to work independently     agree   
12  I would rather go with the flow and see where ...  disagree   
13  I would prioritize building strong co

# CLEAN MEMORY

In [1]:
import torch
import gc

def clear_gpu_memory():
    torch.cuda.empty_cache()

    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj):
                if obj.is_cuda:
                    del obj
        except Exception:
            pass

    gc.collect()

    print(f"GPU memory allocated: {torch.cuda.memory_allocated(0) / 1024**2:.2f} MB")
    print(f"GPU memory cached: {torch.cuda.memory_cached(0) / 1024**2:.2f} MB")

clear_gpu_memory()
torch.cuda.reset_peak_memory_stats()

GPU memory allocated: 0.00 MB
GPU memory cached: 0.00 MB


/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/torch/__init__.py:836: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  return isinstance(obj, torch.Tensor)
/tmp/ipykernel_78/4229153563.py:18: FutureWarning: `torch.cuda.memory_cached` has been renamed to `torch.cuda.memory_reserved`
  print(f"GPU memory cached: {torch.cuda.memory_cached(0) / 1024**2:.2f} MB")
